# 強化学習


## 行動・状態・報酬

エージェントが環境に対して **行動** し、その結果として **状態** と **報酬** を受け取る。

- **行動 (Action)**：エージェントの振る舞い
- **状態 (State)**：エージェントの置かれた状況
- **即時報酬 (Immediate Reward)**：行動の「即時的な」良さ（= 後のことは考えない）。単に報酬とも呼ぶ

- **ステップ**：エージェントが行動し、その結果環境から状態と即時報酬を受け取る1サイクル
- **エピソード**：強化学習で解きたいタスクを開始してから終了するまでの期間。複数のステップの繰り返し。

- **マルコフ性** (Markov property)：「未来の状態 $$s_{t+1}$$ は現在の状態 $$s_t$$ と現在の行動 $$a_t$$ のみを使って記述でき、それよりも過去の状態や行動には依存しない」という性質
- **マルコフ決定過程** (Markov Decision Process, MDP)：
- **方策 (Policy)**：エージェントの行動を決定するルール（関数）

**強化学習を解く = 方策の最適化問題を解く**

> **【NOTE】**
>
> 「取りうる状態・行動の個数が有限である**有限マルコフ決定過程**においては、最適な方策が少なくとも1つは存在する」ということが知られている

## 即時報酬と収益

すぐには大きな即時報酬を得られない行動でも、後々大きな即時報酬につながる可能性がある  
→ **毎ステップでその時の即時報酬が最大となるように行動しても、最終的に得られる即時報酬の総和が最大化できるとは限らない**  
→ ある時間ステップ以降に得られる即時報酬の累計 = **収益 (Return)** $$G_t$$ で評価する

<img width="700" alt="図1" src="https://user-images.githubusercontent.com/13412823/154613038-de25b35b-28ee-40a3-843e-f84af244f049.png">

収益（= 報酬の累計）について、

- 収益の正確な値は、エピソードが終了してみないと分からない
- しかし、エージェントの立場としては、その時々で行動を選択する段階で収益を知りたい

→ 見込みの値を見積もるしかない  
→ 今よりも未来であるほど見積もりは不確かになるので、累計を取る際に **割引率** $$0 \lt \gamma \lt 1$$ をかけて和を取る

$$
G_t
\equiv r_{t+1} +  \gamma r_{t+2} + \gamma^2 r_{t+3} + \cdots
= \displaystyle \sum_{k=0}^{\infty} \gamma^{k} r_{t+k+1}
$$

式変形して再帰的な式で表すこともできる：

$$
G_t = r_{t+1} +  \gamma G_{t+1}
$$



## 状態価値と状態行動価値

前述の見込み収益 $$R_t$$ を計算するには、将来の即時報酬 $$r_{t+1}, r_{t+2}, \cdots$$ の値がわかっている必要がある。  
実際の即時報酬は行動してみないと分からないため、現実的ではない。

→ 確率に基づいて期待値を計算する

$$E$$ で期待値計算を表現すると、状態 $$s$$ における収益の期待値 = **状態価値 (State Value)** は

$$
\begin{eqnarray}
V^\pi(s_t) &\equiv& E \left( r_{t+1} + \gamma r_{t+2} + \gamma^2 r_{t+3} + \cdots \right) \\
&=& E \left( r_{t+1} + \gamma V^{\pi}(s_{t+1}) \right)
\end{eqnarray}
$$

と再帰的に表せる。

期待値は

- 行動選択確率 $$\pi \left( a \middle| s \right)$$：方策 $$\pi$$ のもとで、状態 $$s$$ が与えられたとき、エージェントが行動 $$a$$ を取る確率
- 状態遷移確率 $$P \left( s' \middle| s, a \right)$$：状態 $$s$$ においてエージェントが行動 $$a$$ を取ったとき、状態 $$s'$$ に遷移する確率

をかけて全ての $$a, s'$$ で和を取ることで計算できるから、$$r_{t+1}$$ を「状態 $$s$$ から 状態 $$s'$$ に遷移したときの即時報酬」を表す関数 $$R(s, s')$$ で置き換えて

$$
V^\pi(s) =
\sum_{a} \pi \left( a \middle| s \right) \sum_{s'} P \left(s' \middle| s, a \right)
\left( R(s, s') + \gamma V^{\pi}(s') \right) \\
$$

これを **ベルマン方程式** (Bellman Equation) と呼ぶ。


-------



- **状態価値** $$V^\pi(s)$$：ある状態 $$s$$ から方策 $$\pi$$ に従って行動し続けた場合にどれだけの収益を得られそうかの期待値
- **状態行動価値** $$Q^\pi(s, a)$$：ある状態 $$s$$ である行動 $$a$$ を取り、その後方策 $$\pi$$ に従って行動し続けた場合にどれだけの収益を得られそうかの期待値

将来の即時報酬

遅いタイミングで得られた報酬に減少補正をかけるため、**割引率** $$0 \lt \gamma \lt 1$$ を導入して定式化する：

$$
V^\pi(s) = E \left( r_t + \gamma r_{t+1} + \gamma^2 r_{t+2} + \cdots ; s_t = s \right)
$$

$$
Q^\pi(s, a)
= E \left( r_t + \gamma r_{t+1} + \gamma^2 r_{t+2} + \cdots ; s_t = s, a_t = a \right)
$$


In [6]:
"""
迷路問題を強化学習で解く

xoogxxxx
ooxxooxs
xoxooxxo
ooxooooo
xoooxxox
"""

import numpy as np
from enum import auto, Enum

class Action(Enum):
    RIGHT = auto()
    LEFT = auto()
    UP = auto()
    DOWN = auto()


class State(Enum):
    START = auto()
    GOAL = auto()
    AISLE = auto()
    WALL = auto()


class Environment:
    def __init__(self, maze):
        self.maze = np.array(maze)
        self.size = self.maze.shape
        start = np.where(self.maze == 's')
        goal = np.where(self.maze == 'g')
        self.start = np.array([start[0][0], start[1][0]])
        self.goal = np.array([goal[0][0], goal[1][0]])
    
    def reset(self):
        pass
    

class Agent:
    def __init__(self, env):
        """
        env : 環境
        """
        self.env = env
        self.position = env.start

    def policy(self):
        pass
    
    def move(self):
        pass

def main():
    maze = [['x','o','o','g','x','x','x','x'],
            ['o','o','x','x','o','o','x','s'],
            ['x','o','x','o','o','x','x','o'],
            ['o','o','x','o','o','o','o','o'],
            ['x','o','o','o','x','x','o','x']]
    env = Environment(maze)
    agent = Agent(env)
    print(agent.position)


def R(s):
    if s == 'g':
        return 1
    else:
        return 0


class State:
    def __init__(self):
        pass

class Action:
    def __init__(self):
        pass


main()

[1 7]
